In [1]:
module setup

export model_parameters, grid_parameters

mutable struct model_parameters
    χ1::Float64 # scale of labor disutility
    χ2::Float64 # elasticity of labor supply
    ρ::Float64 # discount rate
    σ::Float64 # IES
    τ_w::Float64 # tax
    
    ζ_N::Float64 # share of workers in non-tradables
    ζ_T::Float64 # share of workers in tradables
    
    r_d::Float64 # interest rate on deposits
    r_l::Float64 # interest rate on loans
    
    κ::Float64 # slope of Phillips curve 
    θ::Float64 # elasticity of substitution between tradables and non-tradables
    η::Float64 # weight on tradables
    α::Float64 # weight on domestic goods in tradables
    θ_g::Float64 # elasticity of substitution between domestic and foreign tradables
    θ_e::Float64 # elasticity of export demand
end


mutable struct grid_parameters
    l_mean::Float64 # mean of log productivity
    l_var::Float64 # variance of log productivvity
    Δλ::Float64 # step size in probability to switch productivity levels
    λ_bar::Float64 # probability to stay at the highest productivity level 
    K::Int64 # number of productivity levels
    labor # productivity grid
    productivity # stacked productivity grid
    trans_matrix # transition matrix
    
    N::Int64 # number of wealth levels
    da::Float64 # step size of the wealth grid
    a_bar::Float64 # hard borrowing constraint
    q_grid # aset grid
    assets # stacked asset grid
end

end

Main.setup

In [2]:
module functions

export u, u_prime, interest, χ, ξ, q_response, print_decomposition, iterate_workers, transition, iterate_ra, transition_ra, results

using Distributions, Plots, LinearAlgebra, Statistics, LaTeXStrings, SparseArrays, CPUTime, SpecialFunctions, SuiteSparse, Distributed, ParallelDataTransfer, Roots

mutable struct results
    wT_sequence
    wN_sequence
    assetT_sequence
    assetN_sequence
    laborT_sequence
    laborN_sequence
    cT_sequence
    cN_sequence
    pF_sequence
    pT_sequence
    pH_sequence
    pN_sequence
    qT_sequence
    qN_sequence
    qF_sequence
    qH_sequence
    qE_sequence
    μ_sequence
    rd_sequence
    qmain_sequence
    gap_N
    gap_T
    gap_m
    excise_sequence
    marg_sequence
    π_sequence
end

function u(x,par)
    σ = par.σ
    if σ > 1.0
        return (x^(1-σ)-1)/(1-σ)
    end
    if σ == 1.0
        return log(x)
    end
end

function u_prime(x,par)
    σ = par.σ
    return x^(-σ)
end

function interest(q,r,par)
    r_l, r_d = par.r_l, par.r_d
    if q < 0
        return (r_l+r-r_d)*q
    end
    if q >= 0
        return r*q
    end
end

function χ(l,par)
    χ1, χ2 = par.χ1, par.χ2
    return χ1*χ2/(1+χ2)*l^(1+1/χ2)
end

function ξ(marg,w,par) #labor income after optimization
    χ1, χ2 = par.χ1, par.χ2
    return (marg/χ1)^χ2 * w^(1+χ2)
end

function q_response(value_ss,distr_ss,w_sequence,r_sequence,lump_sequence,prop,par,par_grid,t_grid)
    dt, L = t_grid
    K, N, da, a_bar = par_grid.K, par_grid.N, par_grid.da, par_grid.a_bar
    q_grid, labor, trans_matrix = par_grid.q_grid, par_grid.labor, par_grid.trans_matrix
    σ, ρ, χ1, χ2, r_l, r_d = par.σ, par.ρ, par.χ1, par.χ2, par.r_l, par.r_d
    productivity, assets = par_grid.productivity, par_grid.assets
    value_prime, value = zeros(K*N), zeros(K*N)
    value_new = zeros(K*N)
    value = value_ss
    A_labor = sparse(kron(trans_matrix,Diagonal(ones(N))))
    A_asset = Tridiagonal(zeros(N*K-1),zeros(N*K),zeros(N*K-1))
    sF_sequence, sB_sequence, labor_running = zeros(K*N,L), zeros(K*N,L), zeros(K*N,L)
    C_running = zeros(K*N,L)
    interest_this = zeros(K*N)
    lump_this, labor_this = zeros(K*N), zeros(N*K)
    χ_this, ξ_this = zeros(K*N), zeros(K*N)
    value_primeF, value_primeB, sF, sB = zeros(K*N), zeros(K*N), zeros(K*N), zeros(K*N)
    vect = zeros(K*N)
    @inbounds for t in reverse(1:1:L)
        interest_this = repeat(map(q->interest(q,r_sequence[t],par),q_grid),K)
        if prop
            lump_this = lump_sequence[t]*productivity/mean(labor)
        end
        if !prop
            lump_this = ones(N*K)*lump_sequence[t]
        end
        
        @views value_primeF[1:end-1] = @views (value[2:end]-value[1:end-1])/da
        @views value_primeB[2:end] = @views (value[2:end]-value[1:end-1])/da
        for num_z in 1:K
            l = find_zero(x->χ1*x^(1/χ2)*(interest_this[end]+lump_this[end]+labor[num_z]*w_sequence[t]*x)^σ-w_sequence[t],1.0)
            c = interest_this[end]+lump_this[end]+labor[num_z]*w_sequence[t]*l
            value_primeF[N*num_z] = c^(-σ)
            l = find_zero(x->χ1*x^(1/χ2)*(interest_this[1]+lump_this[1]+labor[num_z]*w_sequence[t]*x)^σ-w_sequence[t],1.0)
            c = interest_this[1]+lump_this[1]+labor[num_z]*w_sequence[t]*l
            value_primeB[N*(num_z-1)+1] = c^(-σ)
        end
        
        value_primeF = max.(value_primeF,1e-10)
        value_primeB = max.(value_primeB,1e-10)
        labor_thisF = (w_sequence[t]*value_primeF/χ1) .^ χ2
        labor_thisB = (w_sequence[t]*value_primeB/χ1) .^ χ2
        ξ_thisF = w_sequence[t] * productivity .* labor_thisF
        ξ_thisB = w_sequence[t] * productivity .* labor_thisB
        χ_thisF = productivity .* map(l->χ(l,par),labor_thisF)
        χ_thisB = productivity .* map(l->χ(l,par),labor_thisB)

        sF = interest_this + lump_this + ξ_thisF - value_primeF.^(-1/σ)
        sB = interest_this + lump_this + ξ_thisB - value_primeB.^(-1/σ)
        F = Float64.(sF.>0)
        B = Float64.(sB.<0)
        F[N:N:K*N], B[N:N:K*N] = zeros(K), ones(K)
        D = ones(length(F)) - F - B
        cF_this, cB_this = value_primeF.^(-1/σ), value_primeB.^(-1/σ)
        hF = map(c->u(c,par),cF_this) - χ_thisF + sF.*value_primeF
        hB = map(c->u(c,par),cB_this) - χ_thisB + sB.*value_primeB
        both = - min.(D,0)
        unique = B.*(ones(length(F))-F) + F.*(ones(length(B))-B)
        F = F.*unique + both.*Float64.(hF.>hB)
        B = B.*unique + both.*Float64.(hF.<=hB)

        χ_this = χ_thisF.*F + χ_thisB.*B
        ξ_this = ξ_thisF.*F + ξ_thisB.*B
        labor_this = labor_thisF.*F + labor_thisB.*B
        value_prime = value_primeF.*F + value_primeB.*B
        neutral_numbers = findall(z->D[z]>0.99,1:K*N)
        for num in neutral_numbers
            l = find_zero(x->χ1*x^(1/χ2)*(interest_this[num]+lump_this[num]+productivity[num]*w_sequence[t]*x)^σ-w_sequence[t],1.0)
            c = interest_this[num]+lump_this[num]+productivity[num]*w_sequence[t]*l
            value_prime[num] = c^(-σ)
            χ_this[num] = productivity[num]*χ(l,par)
            ξ_this[num], labor_this[num] = w_sequence[t]*l*productivity[num], l
        end
        
        sF = sF.*F
        sB = sB.*B
        sF[N:N:K*N], sB[1:N:(K-1)*N+1] = zeros(K), zeros(K)
        
        @views C_running[:,t] = value_prime.^(-1/σ)
        @views C_running[1:N:N*(K-1)+1,t] = min.(interest_this[1:N:(K-1)*N+1]+ξ_this[1:N:(K-1)*N+1]+lump_this[1:N:(K-1)*N+1],value_prime[1:N:(K-1)*N+1].^(-1/σ))
        @views sF_sequence[:,t] = sF
        @views sB_sequence[:,t] = sB
        @views labor_running[:,t] = labor_this
        
        c_this = value_prime.^(-1/σ)
        vect = map(c->u(c,par),c_this) - χ_this + value/dt + A_labor*value
        @views A_asset[CartesianIndex.(1:K*N-1,2:K*N)] = - sF[1:end-1]/da
        @views A_asset[CartesianIndex.(2:K*N,1:K*N-1)] = sB[2:end]/da
        @views A_asset[CartesianIndex.(1:K*N,1:K*N)] = ones(K*N)*(1/dt+ρ) + sF/da - sB/da
        value_new = A_asset \ vect
        value = copy(value_new)
    end
    
    I_matr = sparse(1:K*N,1:K*N,ones(K*N))
    L_integral, C_integral = zeros(K*N), zeros(K*N)
    @inbounds for t in Int64(ceil(3/dt)):-1:1
        @views A_asset[CartesianIndex.(1:K*N-1,2:K*N)] = sF_sequence[1:end-1,t]/da
        @views A_asset[CartesianIndex.(2:K*N,1:K*N-1)] = - sB_sequence[2:end,t]/da 
        @views A_asset[CartesianIndex.(1:K*N,1:K*N)] = - sF_sequence[:,t]/da + sB_sequence[:,t]/da
        C_integral = (I_matr-A_asset*dt-A_labor*dt) \ (C_running[:,t]*dt+C_integral)
        L_integral = (I_matr-A_asset*dt-A_labor*dt) \ (labor_running[:,t]*dt+L_integral)
    end
        
    return C_integral, L_integral, value
end

function print_decomposition(Call_T,Call_N,par,par_grid,ss_objects)
    Cq_T, Cq_N, distr_T, distr_N = ss_objects
    S = par_grid.N*par_grid.K
    Δ_T, Δ_N = log.(Call_T./Cq_T), log.(Call_N./Cq_N)
    mean_Δ_T, mean_Δ_N = sum(distr_T.*Δ_T), sum(distr_N.*Δ_N)
    var_Δ_T, var_Δ_N = sum(distr_T.*(Δ_T-mean_Δ_T*ones(S)).^2), sum(distr_N.*(Δ_N-mean_Δ_N*ones(S)).^2)
    mean_T, mean_N = sum(distr_T.*log.(Cq_T)), sum(distr_N.*log.(Cq_N))
    cov_T, cov_N = sum(distr_T.*(Δ_T-mean_Δ_T*ones(S)).*(log.(Cq_T)-mean_T*ones(S))), sum(distr_N.*(Δ_N-mean_Δ_N*ones(S)).*(log.(Cq_N)-mean_N*ones(S)))
    sec_Q = sum(distr_T.*log.(Call_T))-sum(distr_N.*log.(Call_N))
    sec = sum(distr_T.*log.(Cq_T))-sum(distr_N.*log.(Cq_N))
    distr = par.ζ_T*distr_T + par.ζ_N*distr_N
    mean_total = sum(par.ζ_T*distr_T.*log.(Cq_T) + par.ζ_N*distr_N.*log.(Cq_N))
    var_total = sum(par.ζ_T*distr_T.* (log.(Cq_T)-mean_total*ones(S)).^2 + par.ζ_N*distr_N.* (log.(Cq_N)-mean_total*ones(S)).^2)
    mean_Q_total = sum(par.ζ_T*distr_T.*log.(Call_T) + par.ζ_N*distr_N.*log.(Call_N))
    var_Q_total = sum(par.ζ_T*distr_T.* (log.(Call_T)-mean_Q_total*ones(S)).^2 + par.ζ_N*distr_N.* (log.(Call_N)-mean_Q_total*ones(S)).^2)
    disp_T, disp_N, bias_T, bias_N, betw = par.ζ_T*var_Δ_T, par.ζ_N*var_Δ_N, 2*par.ζ_T*cov_T, 2*par.ζ_N*cov_N, par.ζ_N*par.ζ_T*(sec_Q^2-sec^2)
    change_var = var_Q_total-var_total
    print("change in variance = ",round(100*change_var/var_total,digits=2),"%: dispersion T = ",round(100*disp_T/change_var,digits=2))
    print("%, dispersion N = ",round(100*disp_N/change_var,digits=2),"%, wealth bias T = ",round(100*bias_T/change_var,digits=2))
    print("%, wealth bias N = ",round(100*bias_N/change_var,digits=2),"%, between = ",round(100*betw/change_var,digits=2),"%")
    return var_Q_total, var_total, change_var, disp_T, disp_N, bias_T, bias_N, betw
end

function iterate_workers(value_ss,distr_ss,w_sequence,r_sequence,lump_sequence,prop,par,par_grid,t_grid)
    dt, L = t_grid
    K, N, da, a_bar = par_grid.K, par_grid.N, par_grid.da, par_grid.a_bar
    q_grid, labor, trans_matrix = par_grid.q_grid, par_grid.labor, par_grid.trans_matrix
    σ, ρ, χ1, χ2, r_l, r_d = par.σ, par.ρ, par.χ1, par.χ2, par.r_l, par.r_d
    productivity, assets = par_grid.productivity, par_grid.assets
    value_prime, value = zeros(K*N), zeros(K*N)
    value_new = zeros(K*N)
    value = value_ss
    A_labor = sparse(kron(trans_matrix,Diagonal(ones(N))))
    A_asset = Tridiagonal(zeros(N*K-1),zeros(N*K),zeros(N*K-1))
    sF_sequence, sB_sequence, labor_running = zeros(K*N,L), zeros(K*N,L), zeros(K*N,L)
    C_running, C_sequence, labor_sequence = zeros(K*N,L), zeros(L), zeros(L)
    interest_this = zeros(K*N)
    distr_running, asset_sequence = zeros(K*N), zeros(L+1)
    distr_sequence = zeros(K*N,L+1)
    distr_running = distr_ss
    lump_this, labor_this = zeros(K*N), zeros(N*K)
    χ_this, ξ_this = zeros(K*N), zeros(K*N)
    value_primeF, value_primeB, sF, sB = zeros(K*N), zeros(K*N), zeros(K*N), zeros(K*N)
    vect = zeros(K*N)
    @inbounds for t in reverse(1:1:L)
        interest_this = repeat(map(q->interest(q,r_sequence[t],par),q_grid),K)
        if prop
            lump_this = lump_sequence[t]*productivity/mean(labor)
        end
        if !prop
            lump_this = ones(N*K)*lump_sequence[t]
        end
        
        @views value_primeF[1:end-1] = @views (value[2:end]-value[1:end-1])/da
        @views value_primeB[2:end] = @views (value[2:end]-value[1:end-1])/da
        for num_z in 1:K
            l = find_zero(x->χ1*x^(1/χ2)*(interest_this[end]+lump_this[N*num_z]+labor[num_z]*w_sequence[t]*x)^σ-w_sequence[t],1.0)
            c = interest_this[end]+lump_this[N*num_z]+labor[num_z]*w_sequence[t]*l
            value_primeF[N*num_z] = c^(-σ)
            l = find_zero(x->χ1*x^(1/χ2)*(interest_this[1]+lump_this[N*(num_z-1)+1]+labor[num_z]*w_sequence[t]*x)^σ-w_sequence[t],1.0)
            c = interest_this[1]+lump_this[N*(num_z-1)+1]+labor[num_z]*w_sequence[t]*l
            value_primeB[N*(num_z-1)+1] = c^(-σ)
        end
        
        value_primeF = max.(value_primeF,1e-10)
        value_primeB = max.(value_primeB,1e-10)
        labor_thisF = (w_sequence[t]*value_primeF/χ1) .^ χ2
        labor_thisB = (w_sequence[t]*value_primeB/χ1) .^ χ2
        ξ_thisF = w_sequence[t] * productivity .* labor_thisF
        ξ_thisB = w_sequence[t] * productivity .* labor_thisB
        χ_thisF = productivity .* map(l->χ(l,par),labor_thisF)
        χ_thisB = productivity .* map(l->χ(l,par),labor_thisB)

        sF = interest_this + lump_this + ξ_thisF - value_primeF.^(-1/σ)
        sB = interest_this + lump_this + ξ_thisB - value_primeB.^(-1/σ)
        F = Float64.(sF.>0)
        B = Float64.(sB.<0)
        F[N:N:K*N], B[N:N:K*N] = zeros(K), ones(K)
        D = ones(length(F)) - F - B
        cF_this, cB_this = value_primeF.^(-1/σ), value_primeB.^(-1/σ)
        hF = map(c->u(c,par),cF_this) - χ_thisF + sF.*value_primeF
        hB = map(c->u(c,par),cB_this) - χ_thisB + sB.*value_primeB
        both = - min.(D,0)
        unique = B.*(ones(length(F))-F) + F.*(ones(length(B))-B)
        F = F.*unique + both.*Float64.(hF.>hB)
        B = B.*unique + both.*Float64.(hF.<=hB)

        χ_this = χ_thisF.*F + χ_thisB.*B
        ξ_this = ξ_thisF.*F + ξ_thisB.*B
        labor_this = labor_thisF.*F + labor_thisB.*B
        value_prime = value_primeF.*F + value_primeB.*B
        neutral_numbers = findall(z->D[z]>0.99,1:K*N)
        for num in neutral_numbers
            l = find_zero(x->χ1*x^(1/χ2)*(interest_this[num]+lump_this[num]+productivity[num]*w_sequence[t]*x)^σ-w_sequence[t],1.0)
            c = interest_this[num]+lump_this[num]+productivity[num]*w_sequence[t]*l
            value_prime[num] = c^(-σ)
            χ_this[num] = productivity[num]*χ(l,par)
            ξ_this[num], labor_this[num] = w_sequence[t]*l*productivity[num], l
        end
        
        sF = sF.*F
        sB = sB.*B
        sF[N:N:K*N], sB[1:N:(K-1)*N+1] = zeros(K), zeros(K)
        
        @views C_running[:,t] = value_prime.^(-1/σ)
        @views C_running[1:N:N*(K-1)+1,t] = min.(interest_this[1:N:(K-1)*N+1]+ξ_this[1:N:(K-1)*N+1]+lump_this[1:N:(K-1)*N+1],value_prime[1:N:(K-1)*N+1].^(-1/σ))
        @views sF_sequence[:,t] = sF
        @views sB_sequence[:,t] = sB
        @views labor_running[:,t] = labor_this
        
        c_this = value_prime.^(-1/σ)
        vect = map(c->u(c,par),c_this) - χ_this + value/dt + A_labor*value
        @views A_asset[CartesianIndex.(1:K*N-1,2:K*N)] = - sF[1:end-1]/da
        @views A_asset[CartesianIndex.(2:K*N,1:K*N-1)] = sB[2:end]/da
        @views A_asset[CartesianIndex.(1:K*N,1:K*N)] = ones(K*N)*(1/dt+ρ) + sF/da - sB/da
        value_new = A_asset \ vect
        value = copy(value_new)
    end
    
    I_matr = sparse(1:K*N,1:K*N,ones(K*N))
    A_labor_trans = A_labor' + I_matr/dt
    asset_sequence[1] = sum(distr_running'*(repeat(q_grid,K)))
    forcurrency_sequence, loan_sequence = zeros(L+1), zeros(L+1)
    distr_sequence[:,1] = distr_running
    @inbounds for t in 1:1:L
        C_sequence[t] = sum(distr_running'*C_running[:,t])
        loan_sequence[t] = (distr_running'*min.(repeat(q_grid,K),0))[1]
        labor_sequence[t] = sum(distr_running.*labor_running[:,t].*productivity)
        
        vect = A_labor_trans*distr_running
        @views A_asset[CartesianIndex.(1:K*N-1,2:K*N)] = @views sB_sequence[2:end,t]/da 
        @views A_asset[CartesianIndex.(2:K*N,1:K*N-1)] = @views - sF_sequence[1:end-1,t]/da
        @views A_asset[CartesianIndex.(1:K*N,1:K*N)] = @views ones(K*N)/dt + sF_sequence[:,t]/da - sB_sequence[:,t]/da

        distr_running = A_asset \ vect
        asset_sequence[t+1] = sum(distr_running'*(repeat(q_grid,K)))
        distr_sequence[:,t+1] = distr_running
    end
    loan_sequence[end] = loan_sequence[end-1]
    
    return asset_sequence, loan_sequence, labor_sequence, C_sequence, C_running, distr_sequence
end

function transition(initial_sequences,ψ,indicators,ϕ_ϵ,par,par_grid,prod_shifters,ss_objects,t_grid,k_limit,dist_limit)
    prod_N, prod_T = prod_shifters
    q_N, q_F, q_H, q_E, q_T, q_total, G, consumption, debt, loan_arbitrage, distr_T, value_T, cons_T, labor_supply_T, distr_N, value_N, cons_N, labor_supply_N = ss_objects
    dt, L = t_grid
    par_grid_N, par_grid_T = deepcopy(par_grid), deepcopy(par_grid)
    par_grid_N.labor = par_grid.labor * prod_N
    par_grid_N.productivity = par_grid.productivity * prod_N
    par_grid_T.labor = par_grid.labor * prod_T
    par_grid_T.productivity = par_grid.productivity * prod_T
    flexible, peg, ner_targeting, taxes_adjust, gov_adjust, flat, prop = indicators
    wT_sequence, wN_sequence, marg_sequence, rd_sequence, excise_sequence, gap_N, gap_T, gap_m, gap_μ = initial_sequences
    r_d, r_l, α, η, θ, θ_g, θ_e, ζ_T, ζ_N, χ2, χ1, σ, τ_w = par.r_d, par.r_l, par.α, par.η, par.θ, par.θ_g, par.θ_e, par.ζ_T, par.ζ_N, par.χ2, par.χ1, par.σ, par.τ_w
    assets = par_grid.assets
    loanN_sequence, loanT_sequence = loan_arbitrage*ones(L+1)/(r_d-r_l), loan_arbitrage*ones(L+1)/(r_d-r_l)
    consumption_sequence, qmain_sequence = zeros(L), zeros(L)
    pF_sequence, pE_sequence, pH_sequence, pT_sequence, pN_sequence = ones(L), ones(L), ones(L), ones(L), ones(L)
    qF_sequence, qE_sequence, qH_sequence, qT_sequence, qN_sequence = ones(L), ones(L), ones(L), ones(L), ones(L)
    μ_sequence, π_sequence = zeros(L), zeros(L)
    cN_running, cT_running = [], []
    laborN_sequence, laborT_sequence = [], []
    assetN_sequence, assetT_sequence = [], []
    cN_sequence, cT_sequence = [], []
    distrN_sequence, distrT_sequence = [], []
    exc_N = prod_N/(prod_N*ζ_N+prod_T*ζ_T)*prop + 1*flat
    exc_T = prod_T/(prod_N*ζ_N+prod_T*ζ_T)*prop + 1*flat
    #exc_N, exc_T = 1, 1
    
    max_T, max_N, max_m = [], [], []
    mean_T, mean_N, mean_m = [], [], []
    first_T, first_N, first_m = [], [], []
    dist_T, dist_N, dist_m, k = 10, 10, 10, 0

    while (k < k_limit) && (max(dist_T,dist_N) > dist_limit)
        k+=1
        print("iteration ",k,"\n")

        # worker block
        @time res = pmap(iterate_workers,[value_N,value_T],[distr_N,distr_T],(1-τ_w)*[wN_sequence,wT_sequence],[rd_sequence,rd_sequence],[-exc_N*excise_sequence,-exc_T*excise_sequence],[prop,prop],[par,par],[par_grid_N,par_grid_T],[t_grid,t_grid])
        assetN_sequence, loanN_sequence, laborN_sequence, cN_sequence, cN_running, distrN_sequence = res[1]
        assetT_sequence, loanT_sequence, laborT_sequence, cT_sequence, cT_running, distrT_sequence = res[2]
        print("workers iterated;\n")

        if flexible
            wT_sequence = wT_sequence + 0.005*gap_T
            wN_sequence = wN_sequence + 0.005*gap_N

            pH_sequence = wT_sequence
            pN_sequence = wN_sequence
            qdom_sequence = ζ_T*laborT_sequence
            qN_sequence = ζ_N*laborN_sequence
            pT_sequence = (ones(L)/η-(1-η)/η*(pN_sequence).^(1-θ)).^(1/(1-θ))
            pE_sequence = wT_sequence
            pF_sequence = (pT_sequence.^(1-θ_g)/(1-α)-α/(1-α)*(pH_sequence).^(1-θ_g)).^(1/(1-θ_g))
            μ_sequence = vcat((pF_sequence[2:end]-pF_sequence[1:end-1])./pF_sequence[1:end-1]/dt,0)
            rd_sequence = r_d*ones(L) + ψ + μ_sequence
            qE_sequence = q_E*(pF_sequence./pE_sequence).^(θ_g)
            qH_sequence = qdom_sequence-qE_sequence
            qF_sequence = (1-α)/α*qH_sequence.*(pH_sequence./pF_sequence).^(θ_g)
            qT_sequence = qH_sequence.*(pH_sequence./pT_sequence).^(θ_g)/α
            qmain_sequence = (η^(1/θ)*qT_sequence.^(1-1/θ)+(1-η)^(1/θ)*qN_sequence.^(1-1/θ)).^(θ/(θ-1))
            nx_sequence = pE_sequence.*qE_sequence - pF_sequence.*qF_sequence
            tax_sequence = τ_w*(wT_sequence.*qdom_sequence+wN_sequence.*qN_sequence) + (loanN_sequence[1:end-1]*ζ_N+loanT_sequence[1:end-1]*ζ_T)*(r_d-r_l)
        end

        if peg
            wT_sequence = wT_sequence + 0.002*gap_T * 4 * 2^(max(dist_T,dist_N)<1e-4)
            wN_sequence = wN_sequence + 0.003*gap_N * 4 * 2^(max(dist_T,dist_N)<1e-4)
            marg_sequence = marg_sequence + 0.05*gap_m * 4 * 2^(max(dist_T,dist_N)<1e-4)

            μ_sequence = par.κ*(ones(L)-marg_sequence)
            π_sequence = - μ_sequence
            pF_sequence = exp.(cumsum(vcat(0,μ_sequence[1:end-1])*dt))
            rd_sequence = r_d*ones(L) + ψ + μ_sequence
            pN_sequence = wN_sequence
            pH_sequence = wT_sequence
            pT_sequence = (α*pH_sequence.^(1-θ_g)+(1-α)*pF_sequence.^(1-θ_g)).^(1/(1-θ_g))
            qdom_sequence = ζ_T*laborT_sequence
            qN_sequence = ζ_N*laborN_sequence
            pE_sequence = wT_sequence        
            qE_sequence = q_E*(pF_sequence./pE_sequence).^(θ_g)
            qH_sequence = qdom_sequence-qE_sequence
            qF_sequence = (1-α)/α*qH_sequence.*(pH_sequence./pF_sequence).^(θ_g)
            qT_sequence = qH_sequence.*(pH_sequence./pT_sequence).^(θ_g)/α
            qmain_sequence = (η^(1/θ)*qT_sequence.^(1-1/θ)+(1-η)^(1/θ)*qN_sequence.^(1-1/θ)).^(θ/(θ-1))
            nx_sequence = pE_sequence.*qE_sequence - pF_sequence.*qF_sequence
            tax_sequence = τ_w*(wT_sequence.*qdom_sequence+wN_sequence.*qN_sequence) + (loanN_sequence[1:end-1]*ζ_N+loanT_sequence[1:end-1]*ζ_T)*(r_d-r_l)
            tax_sequence = tax_sequence + (ones(L)-marg_sequence).*qmain_sequence
        end

        if ner_targeting
            ϕ_π = 1.5*(1-ϕ_ϵ)

            wT_sequence = wT_sequence + 0.005*gap_T
            wN_sequence = wN_sequence + 0.005*gap_N
            marg_sequence = marg_sequence + 0.05*gap_m

            π_sequence = - par.κ*(ones(L)-marg_sequence)
            rd_sequence = r_d*ones(L) + (ϕ_π-1+ϕ_ϵ)/(1-ϕ_ϵ)*π_sequence - ϕ_ϵ/(1-ϕ_ϵ)*ψ
            μ_sequence = (ϕ_π-1+ϕ_ϵ)/(1-ϕ_ϵ)*π_sequence - 1/(1-ϕ_ϵ)*ψ
            pH_sequence = wT_sequence
            pN_sequence = wN_sequence
            pF_sequence = reverse(exp.(cumsum(reverse(-μ_sequence)*dt)))
            pT_sequence = (α*pH_sequence.^(1-θ_g)+(1-α)*pF_sequence.^(1-θ_g)).^(1/(1-θ_g))
            qdom_sequence = ζ_T*laborT_sequence
            qN_sequence = ζ_N*laborN_sequence
            pE_sequence = wT_sequence
            qE_sequence = q_E*(pF_sequence./pE_sequence).^(θ_g)
            qH_sequence = qdom_sequence-qE_sequence
            qF_sequence = (1-α)/α*qH_sequence.*(pH_sequence./pF_sequence).^(θ_g)
            qT_sequence = qH_sequence.*(pH_sequence./pT_sequence).^(θ_g)/α
            qmain_sequence = (η^(1/θ)*qT_sequence.^(1-1/θ)+(1-η)^(1/θ)*qN_sequence.^(1-1/θ)).^(θ/(θ-1))
            nx_sequence = pE_sequence.*qE_sequence - pF_sequence.*qF_sequence
            tax_sequence = τ_w*(wT_sequence.*qdom_sequence+wN_sequence.*qN_sequence) + (loanN_sequence[1:end-1]*ζ_N+loanT_sequence[1:end-1]*ζ_T)*(r_d-r_l)
            tax_sequence = tax_sequence + (ones(L)-marg_sequence).*qmain_sequence
        end

        # fiscal block
        if taxes_adjust
            gov_sequence = G*ones(L)
            proficit_sequence = tax_sequence - gov_sequence
            excise_sequence = rd_sequence*debt - proficit_sequence
        end
        if gov_adjust
            gov_sequence = tax_sequence - rd_sequence*debt
            excise_sequence = zeros(L)
        end

        consumption_sequence = ζ_N*cN_sequence+ζ_T*cT_sequence
        demand_sequence = consumption_sequence+gov_sequence
        qT_new = η * (η * pT_sequence.^(1-θ)+(1-η) * pN_sequence.^(1-θ)).^(θ/(1-θ)) .* pT_sequence.^(-θ) .* demand_sequence
        qH_new = α * qT_new .* (pH_sequence./pT_sequence).^(-θ_g)
        qN_new = (1-η) * (η * pT_sequence.^(1-θ) + (1-η) * pN_sequence.^(1-θ)).^(θ/(1-θ)) .* pN_sequence.^(-θ) .* demand_sequence
        m_new = (η * pT_sequence.^(1-θ)+(1-η) * pN_sequence.^(1-θ)).^(1/(1-θ))
        weights_N, weights_T = zeros(L), zeros(L)
        for t in 1:1:L
            weights_T[t] = sum(distrT_sequence[:,t] .* (par_grid_T.productivity) .* cT_running[:,t].^(-σ*χ2))
            weights_N[t] = sum(distrN_sequence[:,t] .* (par_grid_N.productivity) .* cN_running[:,t].^(-σ*χ2))
        end
        wT_new = χ1/(1-τ_w) * ((qH_new+qE_sequence)/ζ_T ./ weights_T).^(1/χ2)
        wN_new = χ1/(1-τ_w) * (qN_new/ζ_N ./ weights_N).^(1/χ2)
        gap_T = wT_new - wT_sequence
        gap_N = wN_new - wN_sequence
        gap_m = m_new - marg_sequence
        gap_μ = μ_sequence - vcat((pF_sequence[2:end]-pF_sequence[1:end-1])./pF_sequence[1:end-1]/dt,0)
        gap_demand = demand_sequence - qmain_sequence
        dist_T, dist_N, dist_m = maximum(abs.(gap_T)), maximum(abs.(gap_N)), maximum(abs.(gap_m))
        append!(mean_T,mean(abs.(gap_T)))
        append!(mean_N,mean(abs.(gap_N)))
        append!(mean_m,mean(abs.(gap_m)))
        append!(max_T,maximum(abs.(gap_T)))
        append!(max_N,maximum(abs.(gap_N)))
        append!(max_m,maximum(abs.(gap_m)))
        append!(first_T,gap_T[1])
        append!(first_N,gap_N[1])
        append!(first_m,gap_m[1])
        print("\e[91mtradables:\e[0m mean gap = ",mean(abs.(gap_T)),", max gap = ",maximum(abs.(gap_T)),", gap at 0 = ",gap_T[1],"\n")
        print("\e[34mnon-tradables:\e[0m mean gap = ",mean(abs.(gap_N)),", max gap = ",maximum(abs.(gap_N)),", gap at 0 = ",gap_N[1],"\n")
        print("\e[47mmarginal cost:\e[0m mean gap = ",mean(abs.(gap_m)),", max gap = ",maximum(abs.(gap_m)),", gap at 0 = ",gap_m[1],"\n")
    end
    
    outputs = max_T, max_N, max_m, mean_T, mean_N, mean_m, results(wT_sequence,wN_sequence,assetT_sequence,assetN_sequence,laborT_sequence,laborN_sequence,cT_sequence,cN_sequence,pF_sequence,pT_sequence,pH_sequence,pN_sequence,qT_sequence,qN_sequence,qF_sequence,qH_sequence,qE_sequence,μ_sequence,rd_sequence,qmain_sequence,gap_N,gap_T,gap_m,excise_sequence,marg_sequence,π_sequence)
    
return outputs

end
    
function iterate_ra(cons_ss,assets_ss,w_sequence,r_sequence,par,par_grid,t_grid)
    dt, L = t_grid
    λ = 0 # not important for the relevant sequences (consumption and labor)
    labor = mean(par_grid.labor)
    σ, ρ, χ1, χ2 = par.σ, par.ρ, par.χ1, par.χ2
    C_sequence, labor_sequence, asset_sequence = zeros(L), zeros(L), zeros(L)
    C_sequence[end], labor_sequence[end] = cons_ss, (w_sequence[end]*cons_ss^(-σ)/χ1)^χ2
    @inbounds for t in reverse(1:1:L-1)
        C_sequence[t] = C_sequence[t+1]*(1-dt*(r_sequence[t]-ρ)/σ)
        labor_sequence[t] = (w_sequence[t] * C_sequence[t]^(-σ) / χ1)^χ2
    end

    asset_sequence[1] = assets_ss
    saving_sequence = zeros(L)
    @inbounds for t in 1:1:L-1
        saving_sequence[t] = r_sequence[t]*asset_sequence[t] + labor*w_sequence[t]*labor_sequence[t] - C_sequence[t]
        asset_sequence[t+1] = asset_sequence[t] + dt*(saving_sequence[t] + λ*(assets_ss-asset_sequence[t]))
    end

    return asset_sequence, C_sequence, labor_sequence * labor
end
    
function transition_ra(initial_sequences,ψ,indicators,ϕ_ϵ,par,par_grid,prod_shifters,ss_objects,t_grid,k_limit,dist_limit)
    prod_N, prod_T = prod_shifters
    q_N, q_F, q_H, q_E, q_T, q_total, G, consumption, debt, labor_T, labor_N, consumption_T, consumption_N, aN, aT = ss_objects
    labor = mean(par_grid.labor)
    peg, ner_targeting = indicators
    par_grid_T, par_grid_N = deepcopy(par_grid), deepcopy(par_grid)
    par_grid_N.labor, par_grid_T.labor = labor*prod_N, labor*prod_T
    dt, L = t_grid
    wT_sequence, wN_sequence, marg_sequence, rd_sequence, gap_N, gap_T, gap_m, gap_μ = initial_sequences
    r_d, r_l, α, η, θ, θ_g, θ_e, ζ_T, ζ_N, χ2, χ1, σ, τ_w = par.r_d, par.r_l, par.α, par.η, par.θ, par.θ_g, par.θ_e, par.ζ_T, par.ζ_N, par.χ2, par.χ1, par.σ, par.τ_w
    consumption_sequence, excise_sequence, qmain_sequence = zeros(L), zeros(L), zeros(L)
    pF_sequence, pE_sequence, pH_sequence, pT_sequence, pN_sequence = ones(L), ones(L), ones(L), ones(L), ones(L)
    qF_sequence, qE_sequence, qH_sequence, qT_sequence, qN_sequence = ones(L), ones(L), ones(L), ones(L), ones(L)
    μ_sequence, π_sequence = zeros(L), zeros(L)
    laborN_sequence, laborT_sequence = [], []
    assetN_sequence, assetT_sequence = [], []
    cN_sequence, cT_sequence = [], []

    max_T, max_N, max_m = [], [], []
    mean_T, mean_N, mean_m = [], [], []
    first_T, first_N, first_m = [], [], []
    dist_T, dist_N, dist_m, k = 10, 10, 10, 0

    while (k < k_limit) && (max(dist_T,dist_N) > dist_limit)
        k+=1
        if k%200 == 0.0
            print("iteration ",k,"\n")
        end

        # worker block
        res = pmap(iterate_ra,[consumption_N,consumption_T],[aN,aT],(1-τ_w)*[wN_sequence,wT_sequence],[rd_sequence,rd_sequence],[par,par],[par_grid_N,par_grid_T],[t_grid,t_grid])
        assetN_sequence, cN_sequence, laborN_sequence = res[1]
        assetT_sequence, cT_sequence, laborT_sequence = res[2]
        #print("workers iterated;\n")

        if peg
            wT_sequence = wT_sequence + 0.002*gap_T 
            wN_sequence = wN_sequence + 0.003*gap_N 
            marg_sequence = marg_sequence + 0.05*gap_m 

            μ_sequence = par.κ*(ones(L)-marg_sequence)
            π_sequence = - μ_sequence
            pF_sequence = exp.(cumsum(vcat(0,μ_sequence[1:end-1])*dt))
            rd_sequence = r_d*ones(L) + ψ + μ_sequence
            pN_sequence = wN_sequence
            pH_sequence = wT_sequence
            pT_sequence = (α*pH_sequence.^(1-θ_g)+(1-α)*pF_sequence.^(1-θ_g)).^(1/(1-θ_g))
            qdom_sequence = ζ_T*laborT_sequence
            qN_sequence = ζ_N*laborN_sequence
            pE_sequence = wT_sequence        
            qE_sequence = q_E*(pF_sequence./pE_sequence).^(θ_g)
            qH_sequence = qdom_sequence-qE_sequence
            qF_sequence = (1-α)/α*qH_sequence.*(pH_sequence./pF_sequence).^(θ_g)
            qT_sequence = qH_sequence.*(pH_sequence./pT_sequence).^(θ_g)/α
            qmain_sequence = (η^(1/θ)*qT_sequence.^(1-1/θ)+(1-η)^(1/θ)*qN_sequence.^(1-1/θ)).^(θ/(θ-1))
            nx_sequence = pE_sequence.*qE_sequence - pF_sequence.*qF_sequence
            tax_sequence = τ_w*(wT_sequence.*qdom_sequence+wN_sequence.*qN_sequence)
            tax_sequence = tax_sequence + (ones(L)-marg_sequence).*qmain_sequence
        end

        if ner_targeting
            ϕ_ϵ = 0.0
            ϕ_π = 1.5*(1-ϕ_ϵ)

            wT_sequence = wT_sequence + 0.005*gap_T
            wN_sequence = wN_sequence + 0.005*gap_N
            marg_sequence = marg_sequence + 0.05*gap_m

            π_sequence = - par.κ*(ones(L)-marg_sequence)
            rd_sequence = r_d*ones(L) + (ϕ_π-1+ϕ_ϵ)/(1-ϕ_ϵ)*π_sequence - ϕ_ϵ/(1-ϕ_ϵ)*ψ
            μ_sequence = (ϕ_π-1+ϕ_ϵ)/(1-ϕ_ϵ)*π_sequence - 1/(1-ϕ_ϵ)*ψ
            pH_sequence = wT_sequence
            pN_sequence = wN_sequence
            pF_sequence = reverse(exp.(cumsum(reverse(-μ_sequence)*dt)))
            pT_sequence = (α*pH_sequence.^(1-θ_g)+(1-α)*pF_sequence.^(1-θ_g)).^(1/(1-θ_g))
            qdom_sequence = ζ_T*laborT_sequence
            qN_sequence = ζ_N*laborN_sequence
            pE_sequence = wT_sequence
            qE_sequence = q_E*(pF_sequence./pE_sequence).^(θ_g)
            qH_sequence = qdom_sequence-qE_sequence
            qF_sequence = (1-α)/α*qH_sequence.*(pH_sequence./pF_sequence).^(θ_g)
            qT_sequence = qH_sequence.*(pH_sequence./pT_sequence).^(θ_g)/α
            qmain_sequence = (η^(1/θ)*qT_sequence.^(1-1/θ)+(1-η)^(1/θ)*qN_sequence.^(1-1/θ)).^(θ/(θ-1))
            nx_sequence = pE_sequence.*qE_sequence - pF_sequence.*qF_sequence
            tax_sequence = τ_w*(wT_sequence.*qdom_sequence+wN_sequence.*qN_sequence)
            tax_sequence = tax_sequence + (ones(L)-marg_sequence).*qmain_sequence
        end

        gov_sequence = tax_sequence - rd_sequence*debt
        consumption_sequence = ζ_N*cN_sequence + ζ_T*cT_sequence
        demand_sequence = consumption_sequence + gov_sequence
        demand_sequence = demand_sequence - ones(L)*(demand_sequence[end]-q_total)
        qT_new = η * (η * pT_sequence.^(1-θ)+(1-η) * pN_sequence.^(1-θ)).^(θ/(1-θ)) .* pT_sequence.^(-θ) .* demand_sequence
        qH_new = α * qT_new .* (pH_sequence./pT_sequence).^(-θ_g)
        qN_new = (1-η) * (η * pT_sequence.^(1-θ) + (1-η) * pN_sequence.^(1-θ)).^(θ/(1-θ)) .* pN_sequence.^(-θ) .* demand_sequence

        m_new = (η * pT_sequence.^(1-θ)+(1-η) * pN_sequence.^(1-θ)).^(1/(1-θ))
        weights_N, weights_T = zeros(L), zeros(L)
        for t in 1:1:L
            weights_T[t] = sum(prod_T*labor .* cT_sequence[t].^(-σ*χ2))
            weights_N[t] = sum(prod_N*labor .* cN_sequence[t].^(-σ*χ2))
        end
        wT_new = χ1/(1-τ_w) * ((qH_new+qE_sequence)/ζ_T ./ weights_T).^(1/χ2)
        wN_new = χ1/(1-τ_w) * (qN_new/ζ_N ./ weights_N).^(1/χ2)
        gap_T = wT_new - wT_sequence
        gap_N = wN_new - wN_sequence
        gap_m = m_new - marg_sequence
        gap_μ = μ_sequence - vcat((pF_sequence[2:end]-pF_sequence[1:end-1])./pF_sequence[1:end-1]/dt,0)
        gap_demand = demand_sequence - qmain_sequence
        dist_T, dist_N, dist_m = maximum(abs.(gap_T)), maximum(abs.(gap_N)), maximum(abs.(gap_m))
        append!(mean_T,mean(abs.(gap_T)))
        append!(mean_N,mean(abs.(gap_N)))
        append!(mean_m,mean(abs.(gap_m)))
        append!(max_T,maximum(abs.(gap_T)))
        append!(max_N,maximum(abs.(gap_N)))
        append!(max_m,maximum(abs.(gap_m)))
        append!(first_T,gap_T[1])
        append!(first_N,gap_N[1])
        append!(first_m,gap_m[1])
        if k%200 == 0.0
            print("\e[91mtradables: \e[0m",mean(abs.(gap_T))," ",maximum(abs.(gap_T))," ",gap_T[1],"\n")
            print("\e[34mnon-tradables: \e[0m",mean(abs.(gap_N))," ",maximum(abs.(gap_N))," ",gap_N[1],"\n")
            print("\e[47mmarginal cost: \e[0m",mean(abs.(gap_m))," ",maximum(abs.(gap_m))," ",gap_m[1],"\n")
        end
    end

    outputs = max_T, max_N, max_m, mean_T, mean_N, mean_m, results(wT_sequence,wN_sequence,assetT_sequence,assetN_sequence,laborT_sequence,laborN_sequence,cT_sequence,cN_sequence,pF_sequence,pT_sequence,pH_sequence,pN_sequence,qT_sequence,qN_sequence,qF_sequence,qH_sequence,qE_sequence,μ_sequence,rd_sequence,qmain_sequence,gap_N,gap_T,gap_m,excise_sequence,marg_sequence,π_sequence)

    return outputs
end
    

end

Main.functions

In [3]:
module ss_functions

export u, u_prime, interest, χ, ξ, solve_workers, C_period, compute_mpc

using Distributions, Plots, LinearAlgebra, Statistics, LaTeXStrings, SparseArrays, CPUTime, SpecialFunctions, SuiteSparse, Distributed, ParallelDataTransfer, Roots

function u(x,par)
    σ = par.σ
    if σ > 1.0
        return (x^(1-σ)-1)/(1-σ)
    end
    if σ == 1.0
        return log(x)
    end
end

function u_prime(x,par)
    σ = par.σ
    return x^(-σ)
end

function interest(q,par) # interest payments
    r_l, r_d = par.r_l, par.r_d
    if q < 0
        return r_l*q
    end
    if q >= 0
        return r_d*q
    end
end

function χ(l,par) # labor disutility
    χ1, χ2 = par.χ1, par.χ2
    return χ1*χ2/(1+χ2)*l^(1+1/χ2)
end

function ξ(marg,w,par) #labor income after optimization
    χ1, χ2 = par.χ1, par.χ2
    return (marg/χ1)^χ2 * w^(1+χ2)
end

function solve_workers(value,w,par,par_grid)
    CPUtic()
    d_iter = 5.0 # very important, step size for iterations
    K, N, da, a_bar = par_grid.K, par_grid.N, par_grid.da, par_grid.a_bar
    q_grid, labor, trans_matrix = par_grid.q_grid, par_grid.labor, par_grid.trans_matrix
    σ, ρ, χ1, χ2, r_l, r_d = par.σ, par.ρ, par.χ1, par.χ2, par.r_l, par.r_d
    productivity, assets = par_grid.productivity, par_grid.assets
    A_hat = sparse(1:K*N,1:K*N,ones(K*N)*(1/d_iter+ρ))
    k, dist, max_k = 0, 10, 5000
    value_prime, value_new = zeros(K*N), zeros(K*N)
    A_labor = sparse(kron(trans_matrix,Diagonal(ones(N))))
    A_asset = Tridiagonal(zeros(N*K-1),zeros(N*K),zeros(N*K-1))
    interest_this = repeat(map(q->interest(q,par),q_grid),K)
    labor_this = zeros(N*K)
    
    χ_this, ξ_this = zeros(K*N), zeros(K*N)
    value_primeF, value_primeB, sF, sB = zeros(K*N), zeros(K*N), zeros(K*N), zeros(K*N)
    matr2 = sparse(1:N,1:N,zeros(N))
    vect2 = zeros(N)
    while (k<max_k) && (dist>1e-9) && (dist<1e3)
        k+=1

        @views value_primeF[1:end-1] = @views (value[2:end]-value[1:end-1])/da
        @views value_primeB[2:end] = @views (value[2:end]-value[1:end-1])/da
        
        for num_z in 1:K
            l = find_zero(x->χ1*x^(1/χ2)*(r_d*q_grid[end]+labor[num_z]*w*x)^σ-w,[1e-5+max(0,-r_d*q_grid[end]/labor[num_z]/w),25.0])
            c = r_d*q_grid[end]+labor[num_z]*w*l
            value_primeF[N*num_z] = c^(-σ)
            l = find_zero(x->χ1*x^(1/χ2)*(r_l*a_bar+labor[num_z]*w*x)^σ-w,[1e-5+max(0,-r_l*a_bar/labor[num_z]/w),25.0])
            c = r_l*a_bar+labor[num_z]*w*l
            value_primeB[N*(num_z-1)+1] = c^(-σ)
        end

        value_primeF = max.(value_primeF,1e-10)
        value_primeB = max.(value_primeB,1e-10)
        labor_thisF = (w*value_primeF/χ1).^χ2
        labor_thisB = (w*value_primeB/χ1).^χ2
        ξ_thisF = w * productivity .* labor_thisF
        ξ_thisB = w * productivity .* labor_thisB
        χ_thisF = productivity .* map(l->χ(l,par),labor_thisF)
        χ_thisB = productivity .* map(l->χ(l,par),labor_thisB)
        sF = interest_this + ξ_thisF - value_primeF.^(-1/σ)
        sB = interest_this + ξ_thisB - value_primeB.^(-1/σ)
        F = Float64.(sF.>0)
        B = Float64.(sB.<0)
        F[N:N:K*N], B[N:N:K*N] = zeros(K), ones(K)
        D = ones(length(F)) - F - B
        cF_this, cB_this = value_primeF.^(-1/σ), value_primeB.^(-1/σ)
        hF = map(c->u(c,par),cF_this) - χ_thisF + sF.*value_primeF
        hB = map(c->u(c,par),cB_this) - χ_thisB + sB.*value_primeB
        both = - min.(D,0)
        unique = B.*(ones(length(F))-F) + F.*(ones(length(B))-B)
        F = F.*unique + both.*Float64.(hF.>hB)
        B = B.*unique + both.*Float64.(hF.<=hB)

        χ_this = χ_thisF.*F + χ_thisB.*B
        ξ_this = ξ_thisF.*F + ξ_thisB.*B
        labor_this = labor_thisF.*F + labor_thisB.*B
        value_prime = value_primeF.*F + value_primeB.*B
            
        neutral_numbers = findall(z->D[z]>0.99,1:K*N)
        for num in neutral_numbers
            l = find_zero(x->χ1*x^(1/χ2)*(interest_this[num]+productivity[num]*w*x)^σ-w,[1e-5+max(0,-interest_this[num]/productivity[num]/w),25.0])
            c = interest_this[num]+productivity[num]*w*l
            value_prime[num] = c^(-σ)
            χ_this[num] = productivity[num]*χ(l,par)
            ξ_this[num], labor_this[num] = w*l*productivity[num], l
        end
            
        sF = sF.*F
        sB = sB.*B
        sF[N:N:K*N], sB[1:N:(K-1)*N+1] = zeros(K), zeros(K)
        
        c_this = value_prime.^(-1/σ)
        vect = map(c->u(c,par),c_this) - χ_this + value/d_iter + A_labor*value
        @views A_asset[CartesianIndex.(1:K*N-1,2:K*N)] = - sF[1:end-1]/da
        @views A_asset[CartesianIndex.(2:K*N,1:K*N-1)] = sB[2:end]/da
        @views A_asset[CartesianIndex.(1:K*N,1:K*N)] = ones(K*N)*(1/d_iter+ρ) + sF/da - sB/da

        value_new = A_asset \ vect
        dist = mean(abs.(value-value_new))
        value = value_new
    end
    if k == max_k
        print("CONVERGENCE FAILED\n")
    end
    print("distance = ",dist,", ",k," iterations\n")
    CPUtoc();
    matr = sparse(-A_asset+A_labor)+sparse(1:K*N,1:K*N,ones(K*N)*(1/d_iter+ρ))
    matr2 = copy(matr')
    matr2[1,:] = ones(K*N)
    pivot = zeros(K*N)
    pivot[1] = 1
    distr_stat = matr2 \ pivot
    cons_stat = value_prime.^(-1/σ)
    cons_stat[1:N:N*(K-1)+1] = min.(map(q->interest(q,par),assets[1:N:(K-1)*N+1])+ξ_this[1:N:(K-1)*N+1],value_prime[1:N:(K-1)*N+1].^(-1/σ))
    c_quarter = C_period(cons_stat,matr,3)
    c_year = C_period(cons_stat,matr,12)
    return distr_stat, value, cons_stat, c_quarter, c_year, labor_this, sum(labor_this.*productivity.*distr_stat), dist
end

function C_period(cons_stat,matr_stat,period)
    T = 12
    dt = period/T
    integral = zeros(length(cons_stat))
    matr = sparse(1:length(cons_stat),1:length(cons_stat),ones(length(cons_stat))) - dt*matr_stat
    
    for t in 1:1:T
        integral = matr \ (cons_stat*dt+integral)
    end
    return integral
end

function compute_mpc(c_quarter,cons,value,distr,par,par_grid)

    transfer = 1
    productivity = par_grid.productivity

    C = reshape(c_quarter,par_grid.N,par_grid.K)
    MPC = (C[1+transfer:end,:]-C[1:end-transfer,:])/(transfer*par_grid.da)
    MPC_bydecile = (sum(MPC.*reshape(distr,par_grid.N,par_grid.K)[1:end-transfer,:],dims=1)./sum(reshape(distr,par_grid.N,par_grid.K)[1:end-transfer,:],dims=1))'
    v_prime = (reshape(value,par_grid.N,par_grid.K)[2:end,:]-reshape(value,par_grid.N,par_grid.K)[1:end-1,:])/par_grid.da
    matr_income = reshape(productivity,par_grid.N,par_grid.K)[1:end-1,:] .* map(marg->ξ(marg,1-par.τ_w,par),v_prime)
    ΔC_labor = (C[1:end-1,2:end]-C[1:end-1,1:end-1])./(matr_income[:,2:end]-matr_income[:,1:end-1])/3
    ΔC_labor_byasset = sum(reshape(distr,par_grid.N,par_grid.K)[1:end-1,1:end-1].*ΔC_labor,dims=2)./sum(reshape(distr,par_grid.N,par_grid.K)[1:end-1,1:end-1],dims=2)/2
    ΔC_labor_byasset += sum(reshape(distr,par_grid.N,par_grid.K)[1:end-1,2:end].*ΔC_labor,dims=2)./sum(reshape(distr,par_grid.N,par_grid.K)[1:end-1,2:end],dims=2)/2
    conditional_1Yforward = Base.power_by_squaring(par_grid.trans_matrix+Matrix(I,par_grid.K,par_grid.K),12)
    C_bydecile = sum(reshape(c_quarter.*distr,par_grid.N,par_grid.K),dims=1)'
    abs_growth = sum(conditional_1Yforward.*log.(repeat(C_bydecile',par_grid.K,1)./repeat(C_bydecile,1,par_grid.K)),dims=2)
    rel_growth = abs_growth - ones(par_grid.K)*abs_growth[end]

    share = transfer*par_grid.da/(par_grid.assets'*distr)
    neg_wealth = sum(sum(reshape(distr,par_grid.N,par_grid.K),dims=2)[1:findlast(z->z<0,par_grid.q_grid)])
    pos_wealth = sum(sum(reshape(distr,par_grid.N,par_grid.K),dims=2)[findfirst(z->z>0.0,par_grid.q_grid):end])
    print("MPC: ",sum(reshape(distr,par_grid.N,par_grid.K)[1:end-transfer,:].*MPC)," out of ",share,"\n")
    print("negative liquid wealth: ",neg_wealth,"\n")
    print("zero liquid wealth: ",1-neg_wealth-pos_wealth,"\n")
    income_distr = sum(reshape(distr,par_grid.N,par_grid.K),dims=1)
    
    return MPC*income_distr', MPC_bydecile
end

end

Main.ss_functions